In [ ]:
%cd /content/drive/MyDrive/ML/Project1/Pneumonia
%ls

/content/drive/.shortcut-targets-by-id/165/Pneumonia
chest_xray/  __MACOSX/  test/  train/  valid/


In [ ]:
from keras.callbacks import EarlyStopping
from keras.layers import Conv2D, MaxPool2D, Dense, Flatten, Input,Dropout
from keras.models import Sequential
from matplotlib import pyplot as plt
import numpy as np
import os
import cv2
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale = 1./255,
                                   zoom_range = 0.2, 
                                   vertical_flip = True)

training_set = train_datagen.flow_from_directory(directory =  'train',
                                                 batch_size = 8,
                                                 target_size = (150, 150),
                                                 class_mode = 'binary')

test_datagen = ImageDataGenerator(rescale = 1./255)

test_set = test_datagen.flow_from_directory(directory =  'test',
                                                 batch_size = 8,
                                                 target_size = (150, 150),
                                                 class_mode = 'binary')

valid_datagen = ImageDataGenerator(rescale = 1./255)

valid_set = valid_datagen.flow_from_directory(directory =  'valid',
                                                 batch_size = 8,
                                                 target_size = (150, 150),
                                                 class_mode = 'binary')



Found 5216 images belonging to 2 classes.
Found 624 images belonging to 2 classes.
Found 16 images belonging to 2 classes.


In [ ]:
model=Sequential([
        Input(shape=(150,150,3)),
        Dropout(0.2),
        Flatten(),
        Dense(256,activation='relu'),
        Dense(1,activation='sigmoid')        
])

In [ ]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dropout_2 (Dropout)          (None, 150, 150, 3)       0         
_________________________________________________________________
flatten_3 (Flatten)          (None, 67500)             0         
_________________________________________________________________
dense_4 (Dense)              (None, 256)               17280256  
_________________________________________________________________
dense_5 (Dense)              (None, 1)                 257       
Total params: 17,280,513
Trainable params: 17,280,513
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])


In [ ]:
model.fit(training_set, validation_data=valid_set, batch_size=10, epochs=30)

Epoch 1/30
652/652 [==============================] - 2326s 4s/step - loss: 3.1227 - accuracy: 0.7544 - val_loss: 2.7991 - val_accuracy: 0.5625
Epoch 2/30
652/652 [==============================] - 77s 119ms/step - loss: 0.4276 - accuracy: 0.8523 - val_loss: 0.4822 - val_accuracy: 0.7500
Epoch 3/30
652/652 [==============================] - 77s 118ms/step - loss: 0.3367 - accuracy: 0.8594 - val_loss: 0.4579 - val_accuracy: 0.8750
Epoch 4/30
652/652 [==============================] - 76s 117ms/step - loss: 0.3068 - accuracy: 0.8637 - val_loss: 0.8321 - val_accuracy: 0.6250
Epoch 5/30
652/652 [==============================] - 77s 118ms/step - loss: 0.3089 - accuracy: 0.8721 - val_loss: 0.5941 - val_accuracy: 0.8125
Epoch 6/30
652/652 [==============================] - 76s 116ms/step - loss: 0.3536 - accuracy: 0.8454 - val_loss: 0.8852 - val_accuracy: 0.6250
Epoch 7/30
652/652 [==============================] - 76s 116ms/step - loss: 0.3403 - accuracy: 0.8436 - val_loss: 0.5018 - val_acc

In [ ]:
test_results = model.predict(test_set)

In [ ]:
import matplotlib.image as mpimg
# Parameters for our graph; we'll output images in a 4x4 configuration
nrows = 4
ncols = 4

val_normal_dir = os.path.join('valid','NORMAL')
val_pneumonia_dir = os.path.join('valid','PNEUMONIA')
val_normal_pix = [os.path.join(val_normal_dir, fname) 
                for fname in os.listdir(val_normal_dir) if fname!='.DS_Store']
val_pneumonia_pix = [os.path.join(val_pneumonia_dir, fname) 
                for fname in os.listdir(val_pneumonia_dir) if fname!='.DS_Store']

# Set up matplotlib fig, and size it to fit 4x4 pics
fig = plt.gcf()
fig.set_size_inches(ncols*4, nrows*4)

# Show images of normal lung x-rays
for i, img_path in enumerate(val_normal_pix):
    # Get predicted label and covert to normal/pneumonia
    label = test_results[i]
    if label>0.5:
        prediction = 'NORMAL'
    else:
        prediction = 'PNEUMONIA'
    
    # Set up subplot; subplot indices start at 1
    sp = plt.subplot(nrows, ncols, i + 1)
    sp.axis('Off') # Don't show axes (or gridlines)
    sp.set_title(f'Actual:NORMAL | Predicted:{prediction}\n Percent: {label}',fontsize=10)

    plt.imshow(cv2.cvtColor(mpimg.imread(img_path), cv2.COLOR_BGR2RGB))
plt.show()

# Set up matplotlib fig, and size it to fit 4x4 pics
fig = plt.gcf()
fig.set_size_inches(ncols*4, nrows*4)

# Show images of pneumonia lung x-rays
for i, img_path in enumerate(val_pneumonia_pix):
    # Get predicted label and covert to normal/pneumonia
    label = test_results[i-8]
    if label<0.5:
        prediction = 'NORMAL'
    else:
        prediction = 'PNEUMONIA'
    
    # Set up subplot; subplot indices start at 1
    sp = plt.subplot(nrows, ncols, i + 1)
    sp.axis('Off') # Don't show axes (or gridlines)
    sp.set_title(f'Actual:PNEUMONIA | Predicted:{prediction}\n Percent: {label}' ,fontsize=10)

    plt.imshow(cv2.cvtColor(mpimg.imread(img_path), cv2.COLOR_BGR2RGB))
    
plt.show()

In [ ]:
%cd /content/drive/MyDrive/ML/Project1
%ls

In [ ]:
model.save('Dense.h5')